## Preguntas

* ¿Qué relaciones existen entre la edad de los peatones su localización/acción cuando ocurrió el accidente?

* Compare la relación entre género y razones de viaje.

* ¿Cómo influye el equipamiento de seguridad en la gravedad del accidente?

In [1]:
import json
import happybase
from pprint import pprint
from datetime import date

In [2]:
def reconect():
    global connection
    try:
        connection.tables()
    except:
        connection = happybase.Connection('localhost')
        connection.open()
        print("reconected\n")
        
def create_table(name, dict_families):
    if name.encode('UTF-8') in connection.tables():
        connection.delete_table(name.encode('UTF-8'), True)
    connection.create_table(
        name,
        dict_families
    )
        
def drop_database():
    for table in connection.tables():
        connection.delete_table(table, True)
        print(table,"droped")
    print("database drop complete\n")

def add_family_data(table, row_key, column_family, column_names, values):
    column_data = {}
    row_key = row_key.encode('UTF-8')
    for i in range(len(column_names)):
        column = (column_family+':'+column_names[i]).encode('UTF-8')
        value = values[i].encode('UTF-8')
        column_data[column] = value

    table.put(row_key, column_data)

### Creacion de tablas iniciales

In [29]:
reconect()

drop_database()

create_table('actions_location', {'User': dict(), 'Situational': dict(), 'Date': dict()})
create_table('safety_consequences', {'Severity': dict(), 'Situational': dict(), 'User': dict(), 'Date': dict()})
create_table('enviromental_relevance', {'Enviroment': dict(), 'Place': dict(), 'Severity': dict()})

print(connection.tables())

reconected

b'actions_location' droped
b'enviromental_relevance' droped
b'safety_consequences' droped
database drop complete

[b'actions_location', b'enviromental_relevance', b'safety_consequences']


## Ultra Table

In [38]:
ultratable = {}
f = open("Sampled.csv")
items = f.readline().replace("\"","").replace("\n","").replace("\ufeff","").split(",")
items_lenght = len(items)
for item in items:
    ultratable[item] = []
# print(users)

reconect()
actions_location_table = connection.table(b'actions_location')
safety_consequences_table = connection.table(b'safety_consequences')
enviromental_relevance_table = connection.table(b'enviromental_relevance')

# for data in range(2):
#     data = f.readline().replace("\"","").replace("\n","").split(",")
all_lines = f.readlines()
total_lines = len(all_lines)
progress = 0

for data in all_lines:
    if progress % (total_lines/100) == 0:
        print("progress:", progress/float(total_lines)*100)
    data = data.replace("\"","").replace("\n","").split(",")
    dict_data = dict(zip(items, data))

    row_key = str(progress)

    timestamp = '20'+dict_data['an'].zfill(2)+dict_data['mois'].zfill(2)+dict_data['jour'].zfill(2)+dict_data['hrmn']

    if dict_data['year_on'] == '':
        age = 0
    else:
        age = int(dict_data['an']) - int(dict_data['year_on'].split("-")[2])
    
    secu_dict_1 = {'1': 'Belt', '2': 'Helmet', '3': "Children's device", '4': 'Reflective equipment', '9': 'Other'}
    secu_dict_2 = {'2': 'No safety equipment', '3': 'Undetermined', '0': 'Undetermined'}
    if len(dict_data['secu'].replace(".0","")) < 2:
        secu = 'Undetermined'
    elif dict_data['secu'][1] == '1': #si el segundo caracter es 1, si se usaba seguridad
        secu = secu_dict_1[dict_data['secu'].replace(".0","")[0]]
    else:
        secu = secu_dict_2[dict_data['secu'].replace(".0","")[1]]
    
    trajet_dict = {
        '': 'Not specified',
        '0': 'Not specified',
        '1': 'Home - work',
        '2': 'Home - school',
        '3': 'Shopping - Shopping',
        '4': 'Professional use',
        '5': 'Promenade - leisure',
        '9': 'Other'
    }

    # actions_location_table
    add_family_data(actions_location_table, 
                    row_key, 
                    'User',
                    ['age', 'sex'],
                    [str(age), dict_data['sexe']])

    add_family_data(actions_location_table, 
                    row_key, 
                    'Situational',
                    ['loc_ped', 'action_ped', 'trip_reason'],
                    [dict_data['locp'], dict_data['actp'], trajet_dict[dict_data['trajet'].replace(".0","")]])

    add_family_data(actions_location_table, 
                    row_key, 
                    'Date',
                    ['timestamp'],
                    [timestamp])

    # safety_consequences_table
    add_family_data(safety_consequences_table,
                   row_key,
                   'Severity',
                   ['severity'],
                   [dict_data['grav']])

    add_family_data(safety_consequences_table,
                   row_key,
                   'Situational',
                   ['safety_equip'],
                   [secu])

    add_family_data(safety_consequences_table,
                   row_key,
                   'User',
                   ['age', 'user_category'],
                   [str(age), dict_data['catu']])

    add_family_data(safety_consequences_table,
                   row_key,
                   'Date',
                   ['timestamp'],
                   [timestamp])
    
    #enviromental_relevance_table
    add_family_data(enviromental_relevance_table,
                   row_key,
                   'Enviroment',
                   ['atmospheric_conds', 'luminosity', 'surface_cond'],
                   [dict_data['atm_decoded'], dict_data['lum_decoded'], dict_data['surf_decoded']])
    
    add_family_data(enviromental_relevance_table,
                   row_key,
                   'Place',
                   ['road_category', 'plan'],
                   ['int_decoded', 'plan_decoded'])
    
    add_family_data(enviromental_relevance_table,
                   row_key,
                   'Severity',
                   ['severity'],
                   [dict_data['grav']])
    progress += 1

f.close()

print("done")

# for key, data in actions_location_table.scan():
#     print(key, data)
# for key, data in safety_consequences_table.scan():
#     print(key, data)
# for key, data in enviromental_relevance_table.scan():
#     print(key, data)

# users_dumps = json.dumps(users)
# f = open("users_cleaned.json","w")
# f.write(users_dumps)
# f.close()

reconected

progress: 0.0
progress: 1.0
progress: 2.0
progress: 3.0
progress: 4.0
progress: 5.0
progress: 6.0
progress: 7.000000000000001
progress: 8.0
progress: 9.0
progress: 10.0
progress: 11.0
progress: 12.0
progress: 13.0
progress: 14.000000000000002
progress: 15.0
progress: 16.0
progress: 17.0
progress: 18.0
progress: 19.0
progress: 20.0
progress: 21.0
progress: 22.0
progress: 23.0
progress: 24.0
progress: 25.0
progress: 26.0
progress: 27.0
progress: 28.000000000000004
progress: 28.999999999999996
progress: 30.0
progress: 31.0
progress: 32.0
progress: 33.0
progress: 34.0
progress: 35.0
progress: 36.0
progress: 37.0
progress: 38.0
progress: 39.0
progress: 40.0
progress: 41.0
progress: 42.0
progress: 43.0
progress: 44.0
progress: 45.0
progress: 46.0
progress: 47.0
progress: 48.0
progress: 49.0
progress: 50.0
progress: 51.0
progress: 52.0
progress: 53.0
progress: 54.0
progress: 55.00000000000001
progress: 56.00000000000001
progress: 56.99999999999999
progress: 57.99999999999999
progr

In [30]:
# for key, data in enviromental_relevance_table.scan():
#     print(key, data)

